# Apendice 10.2: Uso de Herramientas

- [Leccion](#leccion)
- [Ejercicios](#ejercicios)
- [Area de Experimentacion](#area-de-experimentacion)

## Configuracion

Ejecuta la siguiente celda de configuracion para cargar tu clave API y establecer la funcion auxiliar `get_completion`.

In [ ]:
!pip install anthropic

# Importar la biblioteca de expresiones regulares de Python
import re
import anthropic

# Correccion de ruta para importar hints
import sys, os
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)

# Recuperar las variables API_KEY y MODEL_NAME del almacen de IPython
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# Reescrito para usar MODEL_NAME e incluir stop_sequences
def get_completion(messages, system_prompt="", prefill="",stop_sequences=None):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=messages,
        stop_sequences=stop_sequences
    )
    return message.content[0].text

---

## Leccion

Aunque pueda parecer conceptualmente complejo al principio, el uso de herramientas, tambien conocido como llamada a funciones (function calling), es en realidad bastante simple. Ya conoces todas las habilidades necesarias para implementar el uso de herramientas, que en realidad es solo una combinacion de sustitucion y encadenamiento de prompts.

En ejercicios anteriores de sustitucion, sustituimos texto en los prompts. Con el uso de herramientas, sustituimos resultados de herramientas o funciones en los prompts. Claude no puede literalmente llamar o acceder a herramientas y funciones. En su lugar, hacemos que Claude:
1. Genere el nombre de la herramienta y los argumentos que quiere utilizar
2. Detenga cualquier generacion adicional de respuesta mientras se ejecuta la herramienta
3. Luego volvemos a enviar el prompt con los resultados de la herramienta agregados

La llamada a funciones es util porque expande las capacidades de Claude y le permite manejar tareas mucho mas complejas y de multiples pasos.
Algunos ejemplos de funciones que puedes darle a Claude:
- Calculadora
- Contador de palabras
- Consultas a bases de datos SQL y recuperacion de datos
- API del clima

Puedes hacer que Claude use herramientas combinando estos dos elementos:

1. Un prompt de sistema, en el que le damos a Claude una explicacion del concepto de uso de herramientas asi como una lista descriptiva detallada de las herramientas a las que tiene acceso
2. La logica de control con la que orquestar y ejecutar las solicitudes de uso de herramientas de Claude

### Hoja de ruta del uso de herramientas

*Esta leccion ensena nuestro formato actual de uso de herramientas. Sin embargo, estaremos actualizando y mejorando la funcionalidad de uso de herramientas en el futuro cercano, incluyendo:*
* *Un formato mas simplificado para definiciones y llamadas a funciones*
* *Manejo de errores y cobertura de casos extremos mas robustos*
* *Integracion mas estrecha con el resto de nuestra API*
* *Mejor fiabilidad y rendimiento, especialmente para tareas de uso de herramientas mas complejas*

### Ejemplos

Para habilitar el uso de herramientas en Claude, comenzamos con el prompt de sistema. En este prompt de sistema especial para uso de herramientas, le decimos a Claude:
* La premisa basica del uso de herramientas y en que consiste
* Como Claude puede llamar y usar las herramientas que se le han proporcionado
* Una lista detallada de herramientas a las que tiene acceso en este escenario especifico

Aqui esta la primera parte del prompt de sistema, explicando el uso de herramientas a Claude. Esta parte del prompt de sistema es generalizable a todas las instancias de uso de herramientas con Claude. La estructura de llamada a herramientas que le damos a Claude (`<function_calls> [...] </function_calls>`) es una estructura para la que Claude ha sido especificamente entrenado, por lo que recomendamos que la uses tal cual.

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

Aqui esta la segunda parte del prompt de sistema, que define las herramientas exactas a las que Claude tiene acceso en esta situacion especifica. En este ejemplo, le daremos a Claude una herramienta de calculadora, que toma tres parametros: dos operandos y un operador.

Luego combinamos las dos partes del prompt de sistema.

In [ ]:
system_prompt_tools_specific_tools = """Here are the functions available in JSONSchema format:
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
Calculator function for doing basic arithmetic.
Supports addition, subtraction, multiplication
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>First operand (before the operator)</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>Second operand (after the operator)</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>The operation to perform. Must be either +, -, *, or /</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

Ahora podemos darle a Claude una pregunta que requiere el uso de la herramienta `calculator`. Usaremos `<function_calls\>` en `stop_sequences` para detectar si y cuando Claude llama a la funcion.

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

stop_sequences = ["</function_calls>"]

# Obtener la respuesta de Claude
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

Ahora podemos extraer los parametros de la llamada a funcion de Claude y ejecutar la funcion en nombre de Claude.

Primero definiremos el codigo de la funcion.

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

Luego extraeremos los parametros de la respuesta de llamada a funcion de Claude. Si todos los parametros existen, ejecutamos la herramienta calculadora.

In [ ]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

Ahora que tenemos un resultado, debemos formatearlo correctamente para que cuando lo pasemos de vuelta a Claude, Claude entienda a que herramienta corresponde ese resultado. Hay un formato establecido para esto que Claude ha sido entrenado para reconocer:
```
<function_results>
<result>
<tool_name>{NOMBRE_HERRAMIENTA}</tool_name>
<stdout>
{RESULTADO_HERRAMIENTA}
</stdout>
</result>
</function_results>
```

Ejecuta la celda a continuacion para formatear el resultado de la herramienta anterior en esta estructura.

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

Ahora todo lo que tenemos que hacer es enviar este resultado de vuelta a Claude agregandolo a la misma cadena de mensajes que antes, y listo!

In [ ]:
full_first_response = function_calling_response + "</function_calls>"

# Construir la conversacion completa
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# Imprimir la respuesta de Claude
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- RESULTADO FINAL -------------")
print(final_response)

Felicidades por ejecutar una cadena completa de uso de herramientas de principio a fin!

Ahora, que pasa si le damos a Claude una pregunta que no requiere usar la herramienta proporcionada en absoluto?

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

stop_sequences = ["</function_calls>"]

# Obtener la respuesta de Claude
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

Exito! Como puedes ver, Claude supo no llamar a la funcion cuando no era necesario.

Si deseas experimentar con los prompts de la leccion sin cambiar ningun contenido anterior, desplazate hasta el final del cuaderno de la leccion para visitar el [**Area de Experimentacion**](#area-de-experimentacion).

---

## Ejercicios
- [Ejercicio 10.2.1 - SQL](#ejercicio-1021---SQL)

### Ejercicio 10.2.1 - SQL
En este ejercicio, escribiras un prompt de uso de herramientas para consultar y escribir en la "base de datos" mas pequena del mundo. Aqui esta la base de datos inicializada, que en realidad es solo un diccionario.

In [ ]:
db = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "products": [
        {"id": 1, "name": "Widget", "price": 9.99},
        {"id": 2, "name": "Gadget", "price": 14.99},
        {"id": 3, "name": "Doohickey", "price": 19.99}
    ]
}

Y aqui esta el codigo de las funciones que escriben y leen de la base de datos.

In [ ]:
def get_user(user_id):
    for user in db["users"]:
        if user["id"] == user_id:
            return user
    return None

def get_product(product_id):
    for product in db["products"]:
        if product["id"] == product_id:
            return product
    return None

def add_user(name, email):
    user_id = len(db["users"]) + 1
    user = {"id": user_id, "name": name, "email": email}
    db["users"].append(user)
    return user

def add_product(name, price):
    product_id = len(db["products"]) + 1
    product = {"id": product_id, "name": name, "price": price}
    db["products"].append(product)
    return product

Para resolver el ejercicio, comienza definiendo un prompt de sistema como `system_prompt_tools_specific_tools` de arriba. Asegurate de incluir el nombre y la descripcion de cada herramienta, junto con el nombre, tipo y descripcion de cada parametro para cada funcion. Te hemos proporcionado una estructura inicial a continuacion.

In [ ]:
system_prompt_tools_specific_tools_sql = """
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools_sql

Cuando estes listo, puedes probar tu prompt de sistema de definicion de herramientas con los ejemplos a continuacion. Solo ejecuta la celda de abajo!

In [ ]:
examples = [
    "Add a user to the database named Deborah.",
    "Add a product to the database named Thingo",
    "Tell me the name of User 2",
    "Tell me the name of Product 3"
]

for example in examples:
    message = {
        "role": "user",
        "content": example
    }

    # Obtener e imprimir la respuesta de Claude
    function_calling_response = get_completion([message], system_prompt=system_prompt, stop_sequences=stop_sequences)
    print(example, "\n----------\n\n", function_calling_response, "\n*********\n*********\n*********\n\n")

Si lo hiciste correctamente, los mensajes de llamada a funcion deberian llamar a las funciones `add_user`, `add_product`, `get_user` y `get_product` correctamente.

Para credito extra, agrega algunas celdas de codigo y escribe codigo de analisis de parametros. Luego llama a las funciones con los parametros que Claude te proporcione para ver el estado de la "base de datos" despues de la llamada.

Si quieres ver una posible solucion, ejecuta la celda de abajo!

In [ ]:
from hints import exercise_10_2_1_solution; print(exercise_10_2_1_solution)

### Felicidades!

Felicidades por aprender el uso de herramientas y la llamada a funciones! Ve a la ultima seccion del apendice si deseas aprender mas sobre busqueda y RAG.

---

## Area de Experimentacion

Esta es un area para que experimentes libremente con los ejemplos de prompts mostrados en esta leccion y modifiques los prompts para ver como puede afectar las respuestas de Claude.

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

In [ ]:
system_prompt_tools_specific_tools = """Here are the functions available in JSONSchema format:
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
Calculator function for doing basic arithmetic.
Supports addition, subtraction, multiplication
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>First operand (before the operator)</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>Second operand (after the operator)</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>The operation to perform. Must be either +, -, *, or /</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

stop_sequences = ["</function_calls>"]

# Obtener la respuesta de Claude
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

In [ ]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

In [ ]:
full_first_response = function_calling_response + "</function_calls>"

# Construir la conversacion completa
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# Imprimir la respuesta de Claude
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- RESULTADO FINAL -------------")
print(final_response)

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

stop_sequences = ["</function_calls>"]

# Obtener la respuesta de Claude
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

---

## Uso Moderno de Herramientas (API Nativa)

La API moderna de Anthropic soporta un parametro nativo `tools` que simplifica significativamente el uso de herramientas. En lugar de definir herramientas dentro del prompt de sistema y analizar manualmente las respuestas XML, puedes pasar las definiciones de herramientas directamente como un parametro estructurado en la llamada a la API. Esto proporciona un manejo de errores mas robusto, mejor fiabilidad y una integracion mas limpia.

In [ ]:
# Ejemplo de uso moderno de herramientas con la API nativa
tools = [
    {
        "name": "calculator",
        "description": "Calculadora para operaciones aritmeticas basicas.",
        "input_schema": {
            "type": "object",
            "properties": {
                "first_operand": {"type": "integer", "description": "Primer operando"},
                "second_operand": {"type": "integer", "description": "Segundo operando"},
                "operator": {"type": "string", "description": "Operacion: +, -, *, /"}
            },
            "required": ["first_operand", "second_operand", "operator"]
        }
    }
]

response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1024,
    tools=tools,
    messages=[{"role": "user", "content": "Multiplica 1984135 por 9343116"}]
)

print(response.content)